In [31]:
import praw
from dotenv import load_dotenv
import os
import requests
from datetime import datetime
import pandas as pd

In [4]:
load_dotenv()

SECRET_KEY = os.getenv("SECRET_KEY")
CLIENT_ID = os.getenv("CLIENT_ID")
PASS = os.getenv("PASSWORD")
USER = os.getenv("USER")

### Get Reddit & Subreddit Instance

In [101]:
# Create read only instance of reddit∏

reddit = praw.Reddit( 
    client_id=CLIENT_ID,
    client_secret=SECRET_KEY,
    user_agent=USER
)

reddit.read_only # check instance

True

In [104]:
washingtondc = reddit.subreddit("washingtondc") # get subreddit instance

washingtondc

Subreddit(display_name='washingtondc')

### Search Threads

In [105]:
def search_subreddit(subreddit, query:str, limit:int):

    thread_list = list() # init list for thread data

    for submission in subreddit.search(query=query, limit=limit): # iterate over all threads returned from search

        result_dict = { # put thread data in a dict
                "submission_id":submission.id,
                "title":submission.title,
                "text":submission.selftext,
                "date":datetime.fromtimestamp(submission.created_utc).strftime('%Y-%m-%d %H:%M:%S')
        }

        thread_list.append(result_dict) # append dict to list

    df = pd.DataFrame(thread_list) # turn list into dataframe
    df.set_index("submission_id", inplace=True) # set index to be thread id

    return df

In [109]:
test_df = search_subreddit(washingtondc, 'title:"safe" title:"neighborhood"', 10)

test_df

,title,text,date
submission_id,,,
vmsxc7,How safe is this neighborhood?,"Hey you, yeah you. Stop fucking asking this. Y...",2022-06-28 13:39:48
16hpztv,Where would you buy in DC to feel most safe ab...,Would the northern part of Georgetown/Glover P...,2023-09-13 11:27:16
3j8vju,Van Ness residents say their neighborhood isn'...,,2015-09-01 13:50:07
lsoay5,Recommendation of a safe neighborhood in which...,"Obviously not looking for luxury, would like a...",2021-02-25 22:02:21
ih2u1o,How safe is the Old City/Near-Northeast neighb...,I'm thinking about signing a lease on a place ...,2020-08-26 13:08:38
chffj1,Late 20’s couple moving from CO to DC looking ...,My girlfriend and I have moved to the DMV area...,2019-07-24 18:57:00
rigywa,Considering buying EOTR in Buena Vista/Histori...,Understand there are still higher rates of cri...,2021-12-17 08:02:16
8tcmdd,Safe to live in Carver-Langston Neighborhood,My husband and I are thinking of looking for h...,2018-06-23 16:15:59
1k3768,Is this neighborhood safe? And questions on gr...,Have an offer to join a group house (4 people ...,2013-08-10 09:55:28


### Get all Commments from Threads

In [122]:
def get_comments(id_list:list, reddit):
    
    comments_list = list()

    for id in id_list:
        
        submission = reddit.submission(id=id)

        for comment in submission.comments.list():

            comment_dict = {
                'submission_id': id,
                'author': comment.author.name if comment.author else None,
                'body': comment.body,
                'score': comment.score,
                'created_utc': comment.created_utc
            }   

            comments_list.append(comment_dict)

    df = pd.DataFrame(comments_list)

    return df


In [121]:
get_comments(list(test_df.index), reddit)

,submission_id,author,body,score,created_utc
0,vmsxc7,downvoteyous,cant comment just got murdered in the face due...,611,1.656438e+09
1,vmsxc7,pizzajona,I’m thinking of moving to Capitol Hill on Janu...,55,1.656483e+09
2,vmsxc7,zero_derivation,Is Columbia Heights a bad neighborhood for rat...,148,1.656452e+09
3,vmsxc7,None,I'm living inside the fenced-in perimeter of t...,194,1.656451e+09
4,vmsxc7,BringMeCoffeeOrTea_,I thought everyone who lived in D.C. was murde...,244,1.656439e+09
...,...,...,...,...,...
413,26ozwk,LobbyDizzle,Well then you have a slew of options. Just che...,1,1.401296e+09
414,26ozwk,dcgrump,> I was looking over along H St NE and Florida...,2,1.401289e+09
415,26ozwk,None,Not really. I work for the federal gov and don...,-1,1.401315e+09
416,26ozwk,acforester,The Noma metro stop is over there and union st...,-3,1.401293e+09
